In [0]:

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        
        self.c1 = nn.Conv2d(1, 16, 3) # i/p channel 1 ; o/p channel : 16, Kernel Size= 3X3 , RF=3 
        self.bn1 = nn.BatchNorm2d(16) # Batch Normalization
        self.dropout1 = nn.Dropout(0.05) #Dropout

        self.c2 = nn.Conv2d(16, 16, 3) # i/p channel 16 ; o/p channel : 16, Kernel Size= 3X3 , RF=5 
        self.bn2 = nn.BatchNorm2d(16)# Batch Normalization
        self.dropout2 = nn.Dropout(0.05)#Dropout

        self.c3 = nn.Conv2d(16, 16, 3) # i/p channel 16 ; o/p channel : 16, Kernel Size= 3X3 , RF=7 
        self.bn3 = nn.BatchNorm2d(16)# Batch Normalization
        self.dropout3 = nn.Dropout(0.05)#Dropout

        self.pool1 = nn.MaxPool2d(2, 2) #mAX POOLING  #14
        

        self.c4 = nn.Conv2d(16, 16, 3)# i/p channel 16 ; o/p channel : 16, KerneL Size= 3X3 , RF=16
        self.bn4 = nn.BatchNorm2d(16) # Batch Normalization
        self.dropout4 = nn.Dropout(0.05)#Dropout

        self.c5 = nn.Conv2d(16, 16, 3)# i/p channel 16 ; o/p channel : 16, KerneL Size= 3X3 ,RF=18
        self.bn5 = nn.BatchNorm2d(16)# Batch Normalization
        self.dropout5 = nn.Dropout(0.05)#Dropout

        self.c6 = nn.Conv2d(16, 64, 3) # i/p channel 16 ; o/p channel : 64, KerneL Size= 3X3,RF=20
        self.bn6 = nn.BatchNorm2d(64)# Batch Normalization
        self.dropout6 = nn.Dropout(0.05)#Dropout


        self.avg_pool = nn.AvgPool2d(kernel_size=5) #GAP 
        self.c7= nn.Conv2d(64, 10, 1)# i/p channel 64 ; o/p channel : 10, KerneL Size= 1X1


    def forward(self, x):

        x = self.pool1(self.dropout3(self.bn3(F.relu(self.c3(self.dropout2(self.bn2(F.relu(self.c2(self.dropout1(self.bn1(F.relu(self.c1(x)))))))))))))
        x = self.dropout5(self.bn5(F.relu(self.c5(self.dropout4(self.bn4(F.relu(self.c4(x))))))))

        x=self.dropout6(self.bn6(F.relu(self.c6(x))))

        x = self.avg_pool(x)
        #x = F.relu(self.c7(x))#
        x = self.c7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [13]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,320
       BatchNorm2d-5           [-1, 16, 24, 24]              32
           Dropout-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 16, 22, 22]           2,320
       BatchNorm2d-8           [-1, 16, 22, 22]              32
           Dropout-9           [-1, 16, 22, 22]               0
        MaxPool2d-10           [-1, 16, 11, 11]               0
           Conv2d-11             [-1, 16, 9, 9]           2,320
      BatchNorm2d-12             [-1, 16, 9, 9]              32
          Dropout-13             [-1, 16, 9, 9]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [14]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [17]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print('Epoch Count : ', epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    print('NEXT.....')
   





  0%|          | 0/469 [00:00<?, ?it/s]

Epoch Count :  1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0736, Accuracy: 9788/10000 (97.88%)

NEXT.....
Epoch Count :  2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0207, Accuracy: 9932/10000 (99.32%)

NEXT.....
Epoch Count :  18


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0194, Accuracy: 9940/10000 (99.40%)

NEXT.....
Epoch Count :  19


loss=0.005799800157546997 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 39.58it/s]



Test set: Average loss: 0.0188, Accuracy: 9941/10000 (99.41%)

NEXT.....
